# Partie 5 - Application Metier: Prediction du Defaut de Credit

Ce notebook applique les modeles developpes a un cas d'usage reel dans le domaine bancaire.

**Contenu:**
- Description du domaine d'application
- Analyse exploratoire des donnees
- Entrainement du modele final
- Extraction des regles de decision
- Interpretation et discussion

**Auteur**: Projet Data Mining - Arbres de Decision

## 1. Description du Domaine d'Application

### 1.1 Problematique Metier

Dans le secteur bancaire, l'evaluation du risque de credit est cruciale pour:
- **Minimiser les pertes** dues aux defauts de paiement
- **Optimiser le portefeuille** de credits accordes
- **Respecter les reglementations** (Bale III, etc.)

### 1.2 Objectif

Predire si un client sera en defaut de paiement ("oui") ou non ("non") en fonction de ses caracteristiques.

### 1.3 Variables

- **Variables explicatives (features)**:
  - Caracteristiques du client (statut proprietaire, situation matrimoniale, etc.)
  - Informations financieres (revenu, etc.)

- **Variable cible**:
  - `defaut`: oui/non (classification binaire)

## 2. Importation et Exploration des Donnees

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

In [ ]:
# Chargement des donnees
base_url = 'https://raw.githubusercontent.com/NassimZahri/Data_Mining/main/data/'
df = pd.read_csv(base_url + 'credit_simple.csv')

print("Dimensions du dataset:", df.shape)
print("\nPremiers enregistrements:")
df.head(10)

In [ ]:
# Informations sur le dataset
print("Informations sur les colonnes:")
print("=" * 60)
for col in df.columns:
    print(f"\n{col}:")
    print(f"  - Type: {df[col].dtype}")
    print(f"  - Valeurs uniques: {df[col].nunique()}")
    print(f"  - Valeurs: {df[col].unique().tolist()}")

In [ ]:
# Distribution de la variable cible
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Graphique en barres
target_counts = df['defaut'].value_counts()
colors = ['#27ae60', '#e74c3c']
axes[0].bar(target_counts.index, target_counts.values, color=colors)
axes[0].set_xlabel('Defaut de paiement')
axes[0].set_ylabel('Nombre de clients')
axes[0].set_title('Distribution de la Variable Cible')
for i, v in enumerate(target_counts.values):
    axes[0].text(i, v + 0.1, str(v), ha='center', fontsize=12, fontweight='bold')

# Graphique en camembert
axes[1].pie(target_counts.values, labels=target_counts.index, autopct='%1.1f%%', 
            colors=colors, startangle=90)
axes[1].set_title('Proportion des Defauts')

plt.tight_layout()
plt.show()

In [ ]:
# Analyse par variable categorielle
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
categorical_cols = df.columns[:-1].tolist()  # Toutes sauf 'defaut'

for idx, col in enumerate(categorical_cols[:4]):
    ax = axes[idx // 2, idx % 2]
    
    # Crosstab
    ct = pd.crosstab(df[col], df['defaut'], normalize='index') * 100
    ct.plot(kind='bar', ax=ax, color=['#27ae60', '#e74c3c'])
    
    ax.set_xlabel(col)
    ax.set_ylabel('Pourcentage')
    ax.set_title(f'Taux de defaut par {col}')
    ax.legend(title='Defaut', loc='upper right')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

## 3. Preparation des Donnees et Entrainement

In [ ]:
# Preparation
X = pd.get_dummies(df.drop('defaut', axis=1))
y = df['defaut'].map({'oui': 1, 'non': 0})

# Division
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Features utilisees ({len(X.columns)}):")
for col in X.columns:
    print(f"  - {col}")

In [ ]:
# Entrainement du modele final: Arbre de decision avec profondeur optimale
# On choisit un arbre interpretable pour l'application metier

# Trouver la meilleure profondeur par validation croisee
depths = range(1, 8)
cv_scores = []

for d in depths:
    model = DecisionTreeClassifier(max_depth=d, random_state=42)
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())

best_depth = depths[np.argmax(cv_scores)]
print(f"Meilleure profondeur par validation croisee: {best_depth}")
print(f"Score CV: {max(cv_scores):.2%}")

In [ ]:
# Modele final: Arbre de decision
tree_model = DecisionTreeClassifier(max_depth=best_depth, random_state=42)
tree_model.fit(X_train, y_train)

# Predictions
y_pred = tree_model.predict(X_test)
y_pred_proba = tree_model.predict_proba(X_test)[:, 1]

print("Performance du modele sur l'ensemble de test:")
print("=" * 60)
print(classification_report(y_test, y_pred, target_names=['Non', 'Oui']))

## 4. Visualisation de l'Arbre de Decision Final

In [ ]:
# Visualisation graphique
plt.figure(figsize=(20, 12))
plot_tree(
    tree_model,
    feature_names=X.columns.tolist(),
    class_names=['Non', 'Oui'],
    filled=True,
    rounded=True,
    fontsize=11,
    proportion=True
)
plt.title('Arbre de Decision pour la Prediction du Defaut de Credit', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Extraction des Regles de Decision

In [ ]:
# Regles en format texte
print("Regles de decision extraites:")
print("=" * 60)
tree_rules = export_text(tree_model, feature_names=X.columns.tolist())
print(tree_rules)

In [ ]:
def extract_rules_readable(tree, feature_names, class_names):
    """
    Extrait les regles de l'arbre sous forme lisible pour un non-expert.
    """
    from sklearn.tree import _tree
    
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    rules = []
    
    def recurse(node, path):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            
            # Branche gauche
            recurse(tree_.children_left[node], path + [(name, "<=", threshold)])
            # Branche droite
            recurse(tree_.children_right[node], path + [(name, ">", threshold)])
        else:
            # C'est une feuille
            class_idx = np.argmax(tree_.value[node])
            class_name = class_names[class_idx]
            samples = tree_.n_node_samples[node]
            rules.append((path, class_name, samples))
    
    recurse(0, [])
    return rules


rules = extract_rules_readable(tree_model, X.columns.tolist(), ['Non', 'Oui'])

print("Regles de decision en langage naturel:")
print("=" * 80)
print()

for i, (conditions, prediction, samples) in enumerate(rules, 1):
    rule_text = " ET ".join([f"{c[0]} {c[1]} {c[2]:.2f}" for c in conditions])
    defaut_label = "OUI (defaut)" if prediction == 'Oui' else "NON (pas de defaut)"
    print(f"Regle {i}:")
    print(f"  SI {rule_text}")
    print(f"  ALORS Prediction = {defaut_label} ({samples} exemples)")
    print()

## 6. Importance des Variables

In [ ]:
# Importance des features
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': tree_model.feature_importances_
}).sort_values('Importance', ascending=False)

# Ne garder que les features avec importance > 0
importance_df = importance_df[importance_df['Importance'] > 0]

print("Importance des variables dans la decision:")
print("=" * 60)
importance_df

In [ ]:
# Graphique d'importance
plt.figure(figsize=(10, 6))

colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(importance_df)))
plt.barh(importance_df['Feature'], importance_df['Importance'], color=colors)
plt.xlabel('Importance')
plt.ylabel('Variable')
plt.title('Importance des Variables pour la Prediction du Defaut')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 7. Comparaison avec Random Forest

In [ ]:
# Modele Random Forest pour comparaison
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

print("Performance Random Forest:")
print("=" * 60)
print(classification_report(y_test, y_pred_rf, target_names=['Non', 'Oui']))

In [ ]:
# Comparaison des matrices de confusion
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for idx, (name, y_p) in enumerate([('Arbre de Decision', y_pred), ('Random Forest', y_pred_rf)]):
    cm = confusion_matrix(y_test, y_p)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['Non', 'Oui'], yticklabels=['Non', 'Oui'])
    axes[idx].set_xlabel('Prediction')
    axes[idx].set_ylabel('Vraie valeur')
    axes[idx].set_title(f'Matrice de Confusion - {name}')

plt.tight_layout()
plt.show()

## 8. Discussion et Interpretation

### 8.1 Interpretabilite

**Arbre de decision:**
- Regles claires et comprehensibles
- Peut etre explique a un non-expert (client, regulateur)
- Facilite l'audit et la conformite reglementaire

**Random Forest:**
- Meilleure performance predictive (potentiellement)
- Modele "boite noire" - difficile a expliquer
- Importance des features disponible mais pas les regles

### 8.2 Limites Observees

1. **Taille du dataset**: Avec peu de donnees, les performances sont limitees
2. **Desequilibre des classes**: Peut biaiser les predictions
3. **Variables manquantes**: Le modele ne dispose que de quelques variables
4. **Donnees manquantes**: Non traitees dans cet exemple

### 8.3 Recommandations Metier

1. **Pour l'interpretabilite**: Utiliser l'arbre de decision simple
2. **Pour la performance**: Utiliser Random Forest ou ensemble
3. **Pour la production**: Combiner les deux (arbre pour explication, RF pour decision)

In [ ]:
# Resume final
print("RESUME DU PROJET")
print("=" * 80)
print()
print("Domaine d'application: Prediction du defaut de credit bancaire")
print()
print("Modele final: Arbre de decision (interpretable)")
print(f"  - Profondeur: {best_depth}")
print(f"  - Accuracy: {(y_pred == y_test).mean():.2%}")
print()
print("Variables les plus importantes:")
for _, row in importance_df.head(3).iterrows():
    print(f"  - {row['Feature']}: {row['Importance']:.2%}")
print()
print("Avantages:")
print("  - Regles explicables")
print("  - Conformite reglementaire")
print("  - Decisions auditables")
print()
print("Limites:")
print("  - Dataset de petite taille")
print("  - Variables limitees")
print("  - Peut manquer des patterns complexes")

In [ ]:
print("\nFin du projet - Arbres de Decision et Applications")
print("Tous les notebooks ont ete completes avec succes.")